In [4]:
import aiohttp
import asyncio
import numpy as np
import pickle
import re
import typing as t
import nltk
from nltk.tokenize import word_tokenize
from pymorphy3 import MorphAnalyzer

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\podov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\podov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Test

In [91]:
answers = {}

async def fetch(session: aiohttp.ClientSession, url: str, id):
    async with session.get(url) as response:
        await asyncio.sleep((1 + np.random.sample()) * np.random.randint(1, 4))
        answers[id] = await response.json()


async def main():
    semaphore = asyncio.Semaphore(7)
    
    async def sem_task(task):
        async with semaphore:
            return await task
    
    async with aiohttp.ClientSession() as session:
        tasks = []
        ids = np.arange(2)
        np.random.shuffle(ids)
        for id in ids:
            tasks.append(
                sem_task(fetch(session, f'http://127.0.0.1:10005/test/{id}', id))
            )
        await asyncio.gather(*tasks, return_exceptions=False)

In [92]:
await main()

In [198]:
answers

{0: {'detail': 'Too Many Requests'}, 1: {'detail': 'Too Many Requests'}}

## Not test

In [5]:
async def sema(n_cor: int, tasks: list) -> None:
    semaphore = asyncio.Semaphore(n_cor)

    async def sem_task(task):
        async with semaphore:
            return await task

    tasks = [sem_task(task) for task in tasks]
    await asyncio.gather(*tasks)

In [6]:
res = {}

In [7]:
async def fetch(session: aiohttp.ClientSession, url: str, payload: dict, storage, id):
    async with session.get(url, params=payload) as response:
        await asyncio.sleep(2 * np.random.sample())
        storage[id] = await response.json()

In [8]:
async with aiohttp.ClientSession() as session:
    tasks = []
    for i in range(20):
        url = f'https://api.hh.ru/vacancies'
        payload = {
            'page': i,
            'per_page': 100,
            'text': 'python',
            'area': 113,  # Россия
            'currency': 'RUR',  # Рубль
            'only_with_salary': 'true',
        }
        tasks.append(fetch(session, url, payload, res, i))
    await sema(7, tasks)

In [9]:
res.keys()

dict_keys([2, 6, 3, 5, 1, 0, 9, 4, 8, 7, 10, 13, 15, 16, 12, 17, 11, 18, 14, 19])

In [10]:
for k in res:
    print(res[k]['per_page'], res[k]['pages'], res[k]['page'])

100 20 2
100 20 6
100 20 3
100 20 5
100 20 1
100 20 0
100 20 9
100 20 4
100 20 8
100 20 7
100 20 10
100 20 13
100 20 15
100 20 16
100 20 12
100 20 17
100 20 11
100 20 18
100 20 14
100 20 19


max = 7 запросов / сек

p.s. и уменьшается под конец?

In [32]:
ids = {
    vacancy['id']
    for page in res.values()
    for vacancy in page['items']
}
len(ids)

2000

In [33]:
ids

{'88979575',
 '91380259',
 '90450784',
 '88698076',
 '90995033',
 '90257504',
 '89140172',
 '91237704',
 '89256423',
 '90337384',
 '91317061',
 '89864454',
 '91416723',
 '90679259',
 '90458712',
 '88950611',
 '90413558',
 '90976859',
 '91244556',
 '73330479',
 '85029726',
 '90743254',
 '89719599',
 '91462760',
 '90037351',
 '90613306',
 '90366994',
 '90259619',
 '87615622',
 '90876897',
 '91385560',
 '90738704',
 '87821981',
 '90936501',
 '90947151',
 '86862664',
 '91039294',
 '90897567',
 '91414753',
 '86185294',
 '91357598',
 '90327539',
 '89709008',
 '91356259',
 '91397273',
 '91251641',
 '90101512',
 '88931048',
 '90437173',
 '91313480',
 '90952118',
 '91451860',
 '90240682',
 '91389809',
 '89438298',
 '90749548',
 '90410518',
 '89719594',
 '90452145',
 '90314947',
 '87918938',
 '86615827',
 '86178653',
 '91221490',
 '90398559',
 '87566551',
 '90536244',
 '91268022',
 '90305149',
 '87647929',
 '86979717',
 '91337313',
 '90338679',
 '90949948',
 '90417867',
 '82893736',
 '90389129',

In [34]:
vacancies = {}

In [35]:
ids_to_fetch = ids.copy()
async with aiohttp.ClientSession() as session:
    while len(ids_to_fetch):
        ok_ids = {k for k, v in vacancies.items() if 'id' in v.keys()}
        ids_to_fetch -= ok_ids
        print(f'Ok is {len(ok_ids)}.\t To fetch is {len(ids_to_fetch)}')

        tasks = []
        for id in ids_to_fetch:
            url = f'https://api.hh.ru/vacancies/{id}'
            payload = {}
            tasks.append(fetch(session, url, payload, vacancies, id))
        await sema(7, tasks)

Ok is 0.	 To fetch is 2000
Ok is 635.	 To fetch is 1365
Ok is 968.	 To fetch is 1032
Ok is 1192.	 To fetch is 808
Ok is 1384.	 To fetch is 616
Ok is 1510.	 To fetch is 490
Ok is 1654.	 To fetch is 346
Ok is 1724.	 To fetch is 276
Ok is 1770.	 To fetch is 230
Ok is 1812.	 To fetch is 188
Ok is 1879.	 To fetch is 121
Ok is 1879.	 To fetch is 121
Ok is 1925.	 To fetch is 75
Ok is 1954.	 To fetch is 46
Ok is 1954.	 To fetch is 46
Ok is 1954.	 To fetch is 46
Ok is 1958.	 To fetch is 42
Ok is 1973.	 To fetch is 27
Ok is 1999.	 To fetch is 1
Ok is 2000.	 To fetch is 0


5 + 7 + 4 + 2 = <18

23m

In [36]:
list(vacancies.values())[0]

{'id': '88979575',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Analyst (уклон в Bi)',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': {'from': 2000, 'to': None, 'currency': 'USD', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'allow_messages': True,
 'experience': {'id': 'between1And3', 'name': 'От 1 года до 3 лет'},
 'schedule': {'id': 'fullDay', 'name': 'Полный день'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': None,
 'contacts': None,
 'description': '<p><strong>Finstar Financial Group</strong> – крупный международный холдинг, в который входят компании по всему миру. Основной бизнес – онлайн кредитование до банковского сегмента. Мы – амбициозная, дружная и активная команда, которая работает над современным финансовым сервисом. Сейчас мы фокусируемся на проек

In [37]:
all(x['description'] for x in vacancies.values())

True

In [39]:
path = '../storage/vacancies_2k__11_01_24.pkl'

# with open(path, 'wb') as f:
#     pickle.dump(vacancies, f)

# with open(path, 'rb') as f:
#     vacancies = pickle.load(f)

In [40]:
import pandas as pd

In [42]:
{x['salary']['currency'] for x in vacancies.values()}

{'EUR', 'RUR', 'USD'}

In [51]:
def convert_to_rub(x: float, currency: str) -> float:
    # Курс на 11.01.2024
    match currency:
        case 'RUR':
            return x
        case 'USD':
            return x * 89.39
        case 'EUR':
            return x * 97.87


data = [
    {
        'id': x['id'],
        'description': x['description'],
        'salary': convert_to_rub(
            x=np.mean([value for value in (x['salary']['from'], x['salary']['to']) if value is not None]),
            currency=x['salary']['currency'],
        ),
    }
    for x in vacancies.values()
]

df = pd.DataFrame(data=data, columns=['id', 'description', 'salary'])
df.head()

,id,description,salary
0,88979575,<p><strong>Finstar Financial Group</strong> – ...,178780.0
1,91380259,<p><strong>Кто мы?</strong></p> <p><strong>AdG...,150000.0
2,90633183,<p><strong>Чем предстоит заниматься:</strong><...,250000.0
3,91216567,<p>Компания Brooma является аккредитованной IT...,120000.0
4,91001613,<p>Для основного продукта <strong>Wallcraft</s...,300000.0


In [56]:
df['salary'].describe()

count       2000.000000
mean      165038.528750
std       107103.085843
min        15000.000000
25%        87000.000000
50%       150000.000000
75%       210000.000000
max      1000000.000000
Name: salary, dtype: float64

In [59]:
corpus = df['description'].values.tolist()

In [60]:
corpus[:5]

['<p><strong>Finstar Financial Group</strong> – крупный международный холдинг, в который входят компании по всему миру. Основной бизнес – онлайн кредитование до банковского сегмента. Мы – амбициозная, дружная и активная команда, которая работает над современным финансовым сервисом. Сейчас мы фокусируемся на проектах для стран Юго-Восточной Азии. В связи с динамичным развитием мы ищем в команду нового <strong>BI analyst </strong>с релокацией во Вьетнам.</p> <p><strong>Чем предстоит заниматься:</strong></p> <ul> <li>Взаимодействия с бизнес-заказчиками по вопросам отчетности, анализ требований;</li> <li>Разработка витрин данных, участие в проектировании и развитии архитектуры DWH;</li> <li>Разработка отчетности в Power BI;</li> <li>Автоматизация бизнес-процессов;</li> <li>Написание различной сложности SQL скриптов, оптимизация SQL скриптов;</li> <li>Возможно использование Python в целях автоматизации и организации ETL процессов.</li> </ul> <p><strong>Тебе подойдет вакансия, если у тебя ес

In [61]:
def apply_re(pattern: str, reply_on: str, corpus: t.Sequence[str]) -> t.MutableSequence[str]:
    return list(map(lambda x: re.sub(pattern, reply_on, x), corpus))

In [62]:
corpus = apply_re('(<[^>]+>|\s)', ' ', corpus)
corpus = apply_re('[^{а-яА-Яa-zA-Z0-9}]', ' ', corpus)
corpus = apply_re('\s+', ' ', corpus)

corpus[:10]

[' Finstar Financial Group крупный международный холдинг в который входят компании по всему миру Основной бизнес онлайн кредитование до банковского сегмента Мы амбициозная дружная и активная команда которая работает над современным финансовым сервисом Сейчас мы фокусируемся на проектах для стран Юго Восточной Азии В связи с динамичным развитием мы ищем в команду нового BI analyst с релокацией во Вьетнам Чем предстоит заниматься Взаимодействия с бизнес заказчиками по вопросам отчетности анализ требований Разработка витрин данных участие в проектировании и развитии архитектуры DWH Разработка отчетности в Power BI Автоматизация бизнес процессов Написание различной сложности SQL скриптов оптимизация SQL скриптов Возможно использование Python в целях автоматизации и организации ETL процессов Тебе подойдет вакансия если у тебя есть Математическое техническое высшее образование Опыт работы с Power BI или иным BI инструментом Понимание чем отличается оптимальный SQL запрос от неоптимального и 

In [63]:
morph = MorphAnalyzer()
stop_words = stopwords.words('russian')

corpus = [
    [
        word_norm for word in word_tokenize(doc, language='russian')
        if (word_norm := morph.normal_forms(word)[0]) not in stop_words
    ] for doc in corpus
]

corpus[:5]

[['finstar',
  'financial',
  'group',
  'крупный',
  'международный',
  'холдинг',
  'который',
  'входить',
  'компания',
  'весь',
  'мир',
  'основной',
  'бизнес',
  'онлайн',
  'кредитование',
  'банковский',
  'сегмент',
  'амбициозный',
  'дружный',
  'активный',
  'команда',
  'который',
  'работать',
  'современный',
  'финансовый',
  'сервис',
  'фокусироваться',
  'проект',
  'страна',
  'юго',
  'восточный',
  'азия',
  'связь',
  'динамичный',
  'развитие',
  'искать',
  'команда',
  'новый',
  'bi',
  'analyst',
  'релокация',
  'вьетнам',
  'предстоять',
  'заниматься',
  'взаимодействие',
  'бизнес',
  'заказчик',
  'вопрос',
  'отчётность',
  'анализ',
  'требование',
  'разработка',
  'витрина',
  'данные',
  'участие',
  'проектирование',
  'развитие',
  'архитектура',
  'dwh',
  'разработка',
  'отчётность',
  'power',
  'bi',
  'автоматизация',
  'бизнес',
  'процесс',
  'написание',
  'различный',
  'сложность',
  'sql',
  'скрипт',
  'оптимизация',
  'sql',
  'с

In [65]:
df['description'] = corpus
df.head(3)

,id,description,salary
0,88979575,"[finstar, financial, group, крупный, междунаро...",178780.0
1,91380259,"[adgo, это, молодой, динамично, развивающийся,...",150000.0
2,90633183,"[предстоять, заниматься, поддержка, работоспос...",250000.0


In [68]:
df.to_pickle('../storage/df_vacancies_2k.pkl')

In [220]:
path = '../storage/corpus_vacancies.pkl'

# with open(path, 'wb') as f:
#     pickle.dump(corpus, f)

# with open(path, 'rb') as f:
#     corpus = pickle.load(f)

In [4]:
import pickle
z = [1, 2 ,    3, 4, 5, [1, 2, 2, 1]]
with open('\\\\pns\\data\\a.pkl', 'rb') as f:
    # pickle.dump(z, f)
    print(pickle.load(f))

[1, 2, 3, 4, 5, [1, 2, 2, 1]]
